In [1]:
import sys
sys.path.append('C:/Users/dust/Documents/Github/KorDeBERTa')

In [2]:
import torch
import pytorch_lightning as pl

from tokenizers import Tokenizer
from datasets import IterableDataset
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint

from Data.DataCollator import DataCollatorForSentencePieceSpanMLM
from Model.DebertaV3 import DebertaV3ForPretraining

In [3]:
torch.set_float32_matmul_precision('medium')

In [4]:
SEED=0

In [5]:
pl.seed_everything(SEED)

Global seed set to 0


0

In [6]:
tokenizer = Tokenizer.from_file('../.local/tokenizers/mecab-hf-unigram-880M-128k.json')

In [7]:
mask_id = tokenizer.get_vocab()['[MASK]']
pad_id = tokenizer.get_vocab()['[PAD]']

In [8]:
def gen():
    with open('../../../Preproc5/korean/test.txt', encoding='utf-8-sig') as f:
        for line in f:
            yield line

In [9]:
batch_size = 4
max_steps = 500_000 * (8192//batch_size)

In [10]:
ds = IterableDataset.from_generator(gen)
ds.shuffle(seed=SEED, buffer_size=8_800_000)
dl = DataLoader(ds, batch_size=batch_size, collate_fn=DataCollatorForSentencePieceSpanMLM(tokenizer, truncation_argument={'max_length':512}))

In [11]:
debertav3_pretrainer = DebertaV3ForPretraining('microsoft/deberta-v3-xsmall', mask_id=mask_id, pad_id=pad_id, lr=1e-7, num_warmup_steps=int(max_steps*0.08), num_training_steps=max_steps)

In [12]:
checkpoint_callback = ModelCheckpoint(
    monitor='Loss_D',
    dirpath='../.model/',
    filename='_{epoch:02d}-{Loss_G:.4f}-{Loss_D:.4f}',
    save_weights_only=True,
)

trainer = pl.Trainer(
    accelerator='gpu',
    precision=16,
    max_steps=max_steps,
    # accumulate_grad_batches=256//batch_size,
    # callbacks=[checkpoint_callback],
)

c:\Users\dust\.virtualenvs\KorDeBERTa-GOeAzVPe\lib\site-packages\lightning_fabric\connector.py:555: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\dust\.virtualenvs\KorDeBERTa-GOeAzVPe\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by def

In [13]:
trainer.fit(debertav3_pretrainer,dl)

c:\Users\dust\.virtualenvs\KorDeBERTa-GOeAzVPe\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\dust\.virtualenvs\KorDeBERTa-GOeAzVPe\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name          | Type                            | Params
------------------------------------------------------------------
0 | generator     | DebertaV2ForMaskedLM            | 60.3 M
1 | discriminator | DebertaV2ForTokenClassification | 119 M 
------------------------------------------------------------------
180 M     Trainable params
0         Non-trainable params
180 M 

Training: 0it [00:00, ?it/s]

c:\Users\dust\.virtualenvs\KorDeBERTa-GOeAzVPe\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB (GPU 0; 8.00 GiB total capacity; 4.35 GiB already allocated; 0 bytes free; 6.57 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF